In [16]:
import ee
ee.Initialize()

In [27]:
import sys
sys.path.append(os.path.join(os.getcwd(),'bin'))
from sixs_emulator_ee_sentinel2_batch import SixS_emulator
from atmcorr_input import Atmcorr_input
from atmospheric_correction import atmospheric_correction
from radiance import radiance_from_TOA
from interpolated_LUTs import Interpolated_LUTs

In [18]:
# a place and a mission
geom = ee.Geometry.Point(-157.816222, 21.297481)
mission = 'COPERNICUS/S2'  

In [19]:
# image collection
ic = ee.ImageCollection(mission)\
    .filterBounds(geom)\
    .filterDate('2017-01-01','2017-02-01')\
    .filter(ee.Filter.lt('MEAN_SOLAR_ZENITH_ANGLE',75))

In [28]:
# 6S emulator
se = SixS_emulator(mission)

In [29]:
# interpolate look up tables 
iLUTs = Interpolated_LUTs('COPERNICUS/S2')

In [30]:
# if this is first time will have to download and interpolate
iLUTs.download_LUTs()
iLUTs.interpolate_LUTs()

Extracting zip file..
Done: LUT files available locally
running n-dimensional interpolation may take a few minutes...
iLUT file already exists (skipping interpolation): S2A_MSI_01.lut
iLUT file already exists (skipping interpolation): S2A_MSI_02.lut
iLUT file already exists (skipping interpolation): S2A_MSI_03.lut
iLUT file already exists (skipping interpolation): S2A_MSI_04.lut
iLUT file already exists (skipping interpolation): S2A_MSI_05.lut
iLUT file already exists (skipping interpolation): S2A_MSI_06.lut
iLUT file already exists (skipping interpolation): S2A_MSI_07.lut
iLUT file already exists (skipping interpolation): S2A_MSI_08.lut
iLUT file already exists (skipping interpolation): S2A_MSI_09.lut
iLUT file already exists (skipping interpolation): S2A_MSI_10.lut
iLUT file already exists (skipping interpolation): S2A_MSI_11.lut
iLUT file already exists (skipping interpolation): S2A_MSI_12.lut
iLUT file already exists (skipping interpolation): S2A_MSI_13.lut


In [31]:
# otherwise can just load into the emulator from local files
se.iLUTs = iLUTs.get()

In [32]:
# extract atmcorr inputs as feature collection
# Atmcorr_input.geom = geom  # specify target location (would use image centroid otherwise)
atmcorr_inputs = ic.map(Atmcorr_input.extractor).getInfo()
features = atmcorr_inputs['features']

In [33]:
# atmospherically correct image collection

ic_atmospherically_corrected = []

for feature in features:

    # at-sensor radiance 
    toa = ee.Image(mission+'/'+feature['properties']['imgID'])
    rad = radiance_from_TOA(toa, feature)

    # 6S emulator
    cc = se.run(feature['properties']['atmcorr_inputs'])

    # atmospheric correction
    SR = atmospheric_correction(rad, cc)
    ic_atmospherically_corrected.append(SR)

In [34]:
# Earth Engine image collection
ic_atmospherically_corrected = ee.ImageCollection(ic_atmospherically_corrected)

In [35]:
# test
SR = ee.Image(ic_atmospherically_corrected.first())
print(SR.reduceRegion(ee.Reducer.mean(),geom).getInfo())

{'B8': 0.3079101947591835, 'B12': 0.187208378759739, 'B6': 0.30273065960250845, 'B5': 0.267519227838868, 'B3': 0.23786308414737306, 'B8A': 0.29977697553848054, 'B2': 0.21544715254774705, 'B10': 0.4969368640425421, 'B9': 0.4176718790944925, 'B7': 0.302642586072869, 'B1': 0.13939643278561717, 'B4': 0.26133856162825925, 'B11': 0.2493073375084754}


In [36]:
SR = ee.Image(ic_atmospherically_corrected.first())

from IPython.display import display, Image
region = geom.buffer(5000).bounds().getInfo()['coordinates']
image = Image(url=SR.select(['B4','B3','B2']).getThumbUrl({
                'region':region,
                'min':0,
                'max':0.3
                }))
display(image)